# 运算核与Notebook之间的通信

Jypter Notebook由服务器、客户端与运算核三个部分组成。服务器与客户端由`notebook`模块提供，而通常我们使用的Python运算核由`ipykernel`模块提供。用户使用客户端编写代码，然后发送到运算核执行，再将运算核的结果发送到客户端显示。

本Notebook介绍如何是用`ipykernel`和`notebook`提供的通信功能在客户端与运算核之间实现用户自定义的数据的传输。在后续的Notebook中，我们将使用该功能实现Javascript的绘图库Plotly与运算核之间的通信，从而实现在运算核更新图表的功能。

下面首先在客户端创建一个Textarea文本框，指定其`id`为`jsout`，客户端接收到所有信息将在此文本框中显示。

In [8]:
%%html
<textarea id="jsout" cols=100 rows=10/>

客户端的通信管理对象为`Jupyter.notebook.kernel.comm_manager`，通过它的`register_target()`可以注册一个新的监听频道，当运算核创建该频道的通信对象时，其第二个参数指定的回调函数将被执行。回调函数有两个参数`comm`和`msg`:

* `comm`:为本次通信创建的`Comm`对象，通过该对象可以进行进一步的通信。
* `smg`:开始通信时接收到的消息。

在下面的`register_target()`的回调函数中调用`comm.on_msg()`为此后的通信注册回调函数。在两个回调函数中我们调用`log()`在`id`为`jsout`的文本框中显示`msg`。`msg`为一个JSON对象，这里调用`JSON.stringify()`将其转换为字符串。

In [9]:
%%javascript
var comm_manager=Jupyter.notebook.kernel.comm_manager;
window.recv_messages = [];

var log=function(msg){
    window.recv_messages.push(msg);
    $("#jsout").text( $("#jsout").text() + "\n" + JSON.stringify(msg));
};

var handle_msg=function(msg){
    log(msg);
}

comm_manager.register_target('js-channel', function(comm,msg){
    log(msg);
    comm.on_msg(handle_msg);
})

<IPython.core.display.Javascript object>

接下来在运算核中运行与上面客观端功能相同的代码。为了在客户端显示运算核中接收到的消息，这里使用`ipywidget.Textarea`控件。`ipywidget`模块提供了许多制作界面的控件，这些控件同样也是采用`Comm`对象在客户端与运算核之间同步属性。

In [10]:
from IPython.display import display_javascript
import json
comm_manager = get_ipython().kernel.comm_manager

recv_messages = []

def log(msg):
    ta.value += "\n" + json.dumps(msg)

def on_msg(msg):
    log(msg)
    recv_messages.append(msg)

def on_open(comm, msg):
    log(msg)
    comm.on_msg(on_msg)
    recv_messages.append(msg)

comm_manager.register_target('py-channel', on_open)

from ipywidgets import Textarea

ta = Textarea(width=800, height=200)
ta

下面在运算核中创建与`js-channel`频道通信的`Comm`对象，并调用其`send()`方法发送一条消息到客户端。创建`Comm`对象时发送的消息由`register_target()`的回调函数接收，而`send()`方法发送的消息则由`Comm.on_msg()`的回调函数接收。

请读者运行完下面的代码之后，查看前面`id`为`jsout`的文本框的内容，它将包含两条消息。由于`log()`将接收到的消息写入了全局变量`recv_messages`中，因此还可以在浏览器的命令行调试窗口输入`recv_messages`查看接收到的消息。

In [11]:
from ipykernel.comm import Comm

comm = Comm("js-channel", {"type":"open"})
comm.send({"data":"hello"})

下面在客户端创建一个`Comm`对象，然后调用其`send()`方法发送消息给运算核，在运算核的回调函数中将接收到的消息添加到`Textarea`控件`ta`中：

In [12]:
%%javascript
var comm_manager=Jupyter.notebook.kernel.comm_manager;
var comm = comm_manager.new_comm("py-channel", {type:"open"});
comm.send({"data":"hello"});

<IPython.core.display.Javascript object>

由于我们将运算核接收到的消息添加到了全局变量`recv_messages`中，因此还可以使用下面的语句查看其内容：

In [14]:
print(json.dumps(recv_messages, indent=2))

[
  {
    "buffers": [],
    "content": {
      "data": {
        "type": "open"
      },
      "comm_id": "F530290F44244DA9AE6E39A5045A959B",
      "target_name": "py-channel"
    },
    "msg_type": "comm_open",
    "parent_header": {},
    "msg_id": "A1D0B4BD18074EA689787893C5DE9B4E",
    "header": {
      "version": "5.0",
      "msg_id": "A1D0B4BD18074EA689787893C5DE9B4E",
      "msg_type": "comm_open",
      "username": "username",
      "session": "DBEB7F4BB67742D3A100264C38C4A164",
      "date": "2016-05-01T15:13:58.132719"
    },
    "metadata": {}
  },
  {
    "buffers": [],
    "content": {
      "data": {
        "data": "hello"
      },
      "comm_id": "F530290F44244DA9AE6E39A5045A959B"
    },
    "msg_type": "comm_msg",
    "parent_header": {},
    "msg_id": "22E3ACA8E336444088D872CC8B09DB8A",
    "header": {
      "version": "5.0",
      "msg_id": "22E3ACA8E336444088D872CC8B09DB8A",
      "msg_type": "comm_msg",
      "username": "username",
      "session": "DBEB7F4BB67